### Try to find a way to compute the synchrony of neurons accross time

In [3]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 
import bk.load as bk
import basefunction.anass_load as at

In [27]:
def binSpikes(neurons,binSize = 0.05,start = 0,stop = 0):
    if stop == 0:
        stop = np.max([neuron.as_units('s').index[-1] for neuron in neurons])

    bins = np.arange(start,stop,binSize)
    binned = []
    for neuron in neurons:
        hist,b = np.histogram(neuron.as_units('s').index,bins = bins)
        binned.append(hist)
    return np.array(binned),b

def plotIntervals(intervals,col):
    for interval in intervals.as_units('s').values:
        plt.axvspan(interval[0],interval[1], facecolor=col, alpha=0.5)
        
def synchrony(neurons,metadata,binsize,stru,method = 'corr',plot = True):
    try: bk.session
    except NameError: bk.session = None
    
    binSize = 0.1
    winSize = int(1/binSize)
    step = int(1/binSize)

    
    print(len(neurons))
    print(binsize)
    

    binned,t = binSpikes(neurons,binSize,start = 0,stop = 25000)
    
    stru_shank = {}
    stru_mask  = {}
    
    for s in stru:
            stru_shank.update({s:at.get_structure_shanks(bk.session,s)})
            if not np.any(stru_shank[s]):
                print('Could not compute ' + s + ' because is you don\'t have shank in it')
                continue
            stru_mask.update({s:[i in stru_shank[s] for i in metadata[:,0]]})
    
    if method == 'corr':
        pbar = ProgressBar()
        corr = np.empty((int(binned.shape[1]/step)+1,len(neurons),len(neurons)))   

        for i,j in enumerate(pbar(range(0,binned.shape[1],step))):
            corr[i] = np.abs(np.corrcoef(binned[:,j:j+winSize]))
            corr[i][np.isnan(corr[i])] = 0

        # corr = np.abs(corr)

        stru_corr  = {}
        stru_sync  = {}
            
        for s in stru_mask.keys():
            print('computing '+s)
            stru_corr.update({s:corr[:,:,stru_mask[s]][:,stru_mask[s]]})
            stru_sync.update({s:np.nanmean(stru_corr[s],axis = (1,2))})

        stru_corr.update({'Hpc-BLA':corr[:,:,stru_mask['Hpc']][:,stru_mask['BLA']]})
        stru_sync.update({'Hpc-BLA':np.nanmean(stru_corr['Hpc-BLA'],axis = (1,2))})

        # for s,s2 in itertools.combinations(stru_mask.keys(),2):
        #     print('computing '+s+'-'+s2)
        #     stru_corr.update({s+'-'+s2:corr[:,:,stru_mask[s]][:,stru_mask[s2]]})
        #     stru_sync.update({s+'-'+s2:np.nanmean(stru_corr[s+'-'+s2],axis = (1,2))})

    elif method == 'coactivation':
        #Binarize the binning.
        #count number of cell active at each timebin
        # nb of cell active / total number in da category for each time point. 
    
        binarize_binned = np.where(binned<1,binned,1)       
        stru_sync  = {}
        for s in stru_mask.keys():
            print('computing '+s)
            stru_sync.update({s:np.sum(binarize_binned[stru_mask[s],:],0)})

    else:    

        print(method + ' Is an unknown method')
        raise ValueError('Unknow Method')
    
    
    if plot:
        plt.figure()
        for s in stru_sync:
            plt.plot(t[:-1],savgol_filter(stru_sync[s],2001,3))
        plt.title('synchrony during ' + bk.session)
        plt.legend([s for s in stru_sync.keys()])
        plt.ylabel('r')
        plt.xlabel('Time(s)')

        Rem = nts.IntervalSet(scipy.io.loadmat('States.mat')['Rem'][:,0],scipy.io.loadmat('States.mat')['Rem'][:,1],time_units = 's')
        plotIntervals(Rem,'darkorange')

        sws = nts.IntervalSet(scipy.io.loadmat('States.mat')['sws'][:,0],scipy.io.loadmat('States.mat')['sws'][:,1],time_units = 's')
        plotIntervals(sws,'grey')
    
    return stru_sync

# def main(path):
#     neurons,metadata = bk.loadSpikeData(path)
#     synchrony = synchrony(neurons,metadata,['Hpc','BLA'])
    
#     return neurons,metadata

In [10]:
path = bk.sessions()['Path'][36]
bk.current_session(path)
neurons,metadata = bk.loadSpikeData(bk.path)


Rat : 10 on day : 9
Working with session Rat10-20140701 @ Z:\Rat10\Rat10-20140701
Loading Rat10-20140701.clu.1
Loading Rat10-20140701.clu.2
Loading Rat10-20140701.clu.3
Loading Rat10-20140701.clu.4
Loading Rat10-20140701.clu.5
Loading Rat10-20140701.clu.6
Loading Rat10-20140701.clu.7
Loading Rat10-20140701.clu.8
Loading Rat10-20140701.clu.9
Loading Rat10-20140701.clu.10
Loading Rat10-20140701.clu.11
Loading Rat10-20140701.clu.12
Loading Rat10-20140701.clu.13
Loading Rat10-20140701.clu.14
Loading Rat10-20140701.clu.15
Loading Rat10-20140701.clu.16
Loading Rat10-20140701.clu.17
Loading Rat10-20140701.clu.18
Loading Rat10-20140701.clu.19
Loading Rat10-20140701.clu.20
40
0.1
aze Is an unknown method


ValueError: Unknow Method

In [29]:
syn = synchrony(neurons,metadata,0.1,['Hpc','BLA'],method = 'coactivation')

40
0.1
computing Hpc
computing BLA


In [28]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow(stru_corr['Hpc'][21000,:,:])
plt.clim(-1,1)
plt.subplot(1,3,2)
plt.imshow(np.abs(stru_corr['Hpc'][21000,:,:]))
plt.clim(-1,1)
plt.subplot(1,3,3)
plt.colorbar()

NameError: name 'stru_corr' is not defined

In [16]:
plt.figure()
plt.eventplot(a[stru_mask['Hpc']],color = 'black')
plt.ylabel('Neurons')
plt.xlabel('Time(s)')

NameError: name 'a' is not defined

In [361]:
neurons[1].as_units('s').index[0:3]
a = []
for i in neurons:
    a.append(i.as_units('s').loc[5:8].index)
    
a = np.array(a,dtype = 'object')

In [334]:
# for i in corr:
#     plt.clf()
#     sns.heatmap(i,square = True,vmin = 0,vmax = 0.5,cmap = 'viridis')
#     plt.pause(0.001)


In [10]:
for i in range(0,binned.shape[1],step):
    plt.clf()
    corr = np.corrcoef(binned[:,i:i+winSize])
    corr[np.isnan(corr)] = 0
    sns.heatmap(corr,square = True,vmin = 0,vmax = 0.1,cmap = 'viridis')
    plt.title(i)
    plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyboardInterrupt: 